# CHEM60 - Assignment #16 - Ozone and ODEs

It's not a pun. It's just what it is. Make sure you have read the [ozone and ODEs context notebook](https://drive.google.com/file/d/1YSXKAqCE84XtYABq4O9r6wxBd_3yrTzR/view?usp=sharing) first.

To get started, click on '**File**' in the left menu, then '**Save a copy in Drive**' to ensure you are editing *your* version of this assignment (if you don't, your changes won't be saved!). Then, share the notebook with Prof Kavassalis (skavassalis@g.hmc.edu). This will make it easier for us to look at the notebook if you need help after class.  


# Imports

If you need to add imports, come back and add them here.

In [ ]:
# Standard library imports
import math as m

# Third party imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# This part of the code block is telling matplotlib to make certain font sizes exra, extra large by default
# Here is where I list what parametres I want to set new defaults for
params = {'legend.fontsize': 'xx-large',
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'xx-large',
         'ytick.labelsize':'xx-large'}
# This line updates the default parameters of pyplot (to use our larger fonts)
plt.rcParams.update(params)

Let's load the data you need for your rate constants. If this is unfamilar, go to ozone notebook before you go further.

In [ ]:
Chapman_df = pd.DataFrame()
# download a .txt file containing photolysis rate constants for O2 -> O + O from 0 - 50km~
Chapman_df['J1'] = np.loadtxt(fname = "http://kavassalis.space/s/J1.txt")
# Chapman_df['J1'] = [2.27E-34,6.23E-33,1.75E-31,2.11E-28,2.04E-26,1.94E-24,1.84E-22,1.17E-20,1.11E-19,1.16E-18,2.40E-17,8.01E-16,8.25E-15,7.05E-14,7.88E-13,4.46E-12,1.08E-11,3.94E-11,8.29E-11,1.66E-10,3.46E-10,4.54E-10,5.28E-10,5.72E-10]

# download a .txt file containing photolysis rate constants for O3 -> O2 + O(1D) from 0 - 50km~
Chapman_df['J3'] = np.loadtxt(fname = "http://kavassalis.space/s/J3.txt")
# Chapman_df['J3'] = [2.01E-05,2.25E-05,2.45E-05,2.73E-05,2.85E-05,2.94E-05,3.00E-05,3.06E-05,3.09E-05,3.15E-05,3.28E-05,3.58E-05,3.93E-05,4.60E-05,6.56E-05,1.02E-04,1.40E-04,2.61E-04,4.23E-04,7.63E-04,2.19E-03,4.01E-03,5.81E-03,7.04E-03]

AIRS_df = pd.read_csv('https://kavassalis.space/s/g4averagedAIRS3STD_7_0_Temperature_A20220208-20220208118W_33N_117W_34N.csv',header=8)
# and now I remember it didn't come with altitude
R = 8.31; Ma = 29; g = 9.8
Chapman_df['Altitude'] = [-m.log(AIRS_df['StdPressureLev'][z]/AIRS_df['StdPressureLev'][0])*(R*AIRS_df['AIRS3STD_7_0_Temperature_A'][z])/(Ma*g) for z in range(len(AIRS_df['StdPressureLev']))]
Chapman_df['k2'] = 6E-34*(AIRS_df['AIRS3STD_7_0_Temperature_A']/300)**-2.3
Chapman_df['k4'] = 8E-12*m.exp(1)**(-2060/AIRS_df['AIRS3STD_7_0_Temperature_A'])

Av = 6.023e23 # molecules mol-1 is Avogadro's number
Chapman_df['Na'] = Av*100*AIRS_df['StdPressureLev']/(R*AIRS_df['AIRS3STD_7_0_Temperature_A'])/(100**3)
Chapman_df['O2'] = 0.21*Chapman_df['Na'] # this one is easy, because oxygen is ~21% of the atmosphere

# Q1. Any collaborators? (0 marks)

Working together is encouraged. If you worked with a classmate or two, list their names here.


---



**names maybe?**



---



# Q2. Write your own Runge Kutta function for the Chapman Mechanism **and** test it for a given height  (15 marks).

What's the Chapman mechanis,, you say? No really, check out the ozone notebook first.

The chemical mechanism is here:

\begin{align}
O_{2} + h\nu & \rightarrow O + O(^{1}D) & (\lambda<240nm) &&(R1) \\
O + O_{2} + M & \rightarrow O_{3} + M &  &&(R2) \\
O_{3} + h\nu&  \rightarrow O_{2} + O(^{1}D) & (\lambda<320nm) &&(R3) \\
O_{3} + O& \rightarrow 2O_{2} &  &&(R4)
\end{align}

But, you can simplify things further and only worry about solving these two equations:

\begin{align}
\frac{d[O_3]}{dt} &= k_2[O][O_2][M] - J_3[O_3] - k_4[O_3][O]\\
\frac{d[O]}{dt} &= 2 J_1[O_2] - k_2[O][O_2][M]- k_4[O_3][O]\\
\end{align}

$[O_2]$ is in such excess compared to ozone and atomic oxygen that you can treat it as a constant (for a given height).



**Your function needs to take either the rate constants and $[O_2]$ and $[M]$ for each height** or the height index and then solve for them within the function.

ie.
`def Runge_Kutta_for_Chapman(O3_init, O_init, J1, k2, J3, k4, O2, M, dt, t_max)`

 or

`def Runge_Kutta_for_Chapman(O3_init, O_init, z, dt, t_max)`

**It needs to return an array of ozone and O concentrations.** Then, you need to test that it works by **creating a plot of simulated ozone concentrations for a given height**. This will be important so you can visually confirm that you are running your simulation long enough to reach a steady-state concentration. You likely will need to try several timesteps to find the right now. The finished version should make clear what time step it works best with (the largest one that actually makes a nice looking simulation).

Full marks require an efficient function - ie. don't do wasteful calculations (like you practiced at the end of the in-class notebook).

**Helpful hint**: Is your integration exploding? ie. you get super huge or (positive or negative) values or it just fails? Try making your time step much, much smaller. The timestep has to be small enough that any change in concentration is small and well approximated by the linear assumption we are making. You may want to use what you know from the ozone notebook to pick initial conditions near the eventual steady-state so you don't have to run it as long. If you run out of time to get your simulation to the eventual steady-state, make a note of it.

In [ ]:
# code!



---



#Q3. Now create a plot of steady-state ozone concentrations (as found with your Runge-Kutta solver) vs altitude (5 marks)

This means running your function in a loop for the different altitudes represented in the above data. Assuming you confirmed above your simulation was reaching steady-state, you can just take the final concentration for each height (ie. O3_reult[-1]). It should look very much like the figure in the context notebook under the text that says, "Now plot the final concentration against height". If your above calculations didn't reach steady-state, make sure you still run each height for the same length of time so you can still make a vertical profile. It'll look different than the steady-state profile, but should still give you the ability to see differences at each height.


---





---



#Q Bonus: Want to add more chemistry??

You can! If you scroll to section 10.2 in this classic atmospheric chemistry [text](https://projects.iq.harvard.edu/files/acmg/files/intro_atmo_chem_bookchap10.pdf), you will see the catalytic loss cycles that are needed to make our simulation more realistic. This will require thinking about what rate constants will be needed for these reactions and some initial guesses for concentration of $[H_2O]$, $[NO]$, etc. (to make your life simple, you can approximate those and say $[H_2O] = 10^{-3} \times [O_2]$,
$[NO] = 10^{-10} \times [O_2]$, $[Cl] = 10^{-10} \times [O_2]$ (if you really want more realistic values or more things, just ask!). The more chemistry you add, the closer the simulation will be to the observations (but with anything computational, there is a trade-off - it'll take way longer to solve the system of equations if you add absolutely everything).

In [ ]:
# maybe code for fun???

# Q Bonus 2: Why is this so slow but the example in the ozone context notebook was fast to run?

What might be different about how the function you wrote works from the one in the library we called in the ozone context notebook? Pre-built solvers in Python are much, much faster than ones you could implement here (even if you wrote the method the prebuild solver was using, which is called 'backwards differentiation formula'), but that method has some real advantages too.

**text?**

#Q4. What is missing for you? (1 mark)

Hello, how are you doing this week? What didn't make sense (*and* that you *want* to understand more deeply)?

---



**your answer**



---



# Submit your notebook

It's time to download your notebook and submit it on Canvas. Go to the File menu and click **Download** -> **Download .ipynb**


Then, go to **Canvas** and **submit your assignment** on the assignment page. It should look something like this:

Once it is submitted, you are done!